# BUILDING NLP WEB APPS WITH GRADIO AND HUGGING FACE TRANSFORMERS

What's interesting about the latest version of Gradio is that you can load multiple transformer models in one web app, either in parallel or in series, ie, compare different translation models in one app, or combine translation and summary models in one app.

This is a really useful feature, given the rapidly growing number of transformer models available within as well as across different tasks. Within summarization, for instance, there are [at least 248 models](https://huggingface.co/models?pipeline_tag=summarization) on Hugging Face's model hub. How do you know which one works better for your use case? Well, one way is to directly compare how one performs against another when given the same text to summarize.

Let's try that in this notebook.

In [2]:
import gradio as gr
import re
import warnings


from gradio.mix import Parallel
from nltk.tokenize import sent_tokenize
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSeq2SeqLM,
    Wav2Vec2ForCTC,
    Wav2Vec2Tokenizer,
    pipeline,
)

warnings.filterwarnings('ignore')

# 1. DEFINE TEXT CLEANING AND SUMMARIZATION FUNCTIONS 

In [3]:
# Tweak the text cleaning function further if you wish

def clean_text(text):
    text = text.encode("ascii", errors="ignore").decode(
        "ascii"
    )  # remove non-ascii, Chinese characters
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"\t", " ", text)
    text = text.strip(" ")
    text = re.sub(
        " +", " ", text
    ).strip()  # get rid of multiple spaces and replace with a single
    return text   

## 1.1 SUMMARIZATION VIA HUGGING FACE PIPELINE

Hugging Face's [pipeline allows you to load up several summarization models](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.SummarizationPipeline), from FB's Bart to Google's T5. For this example, I'll use the "facebook/bart-large-cnn" model, which has given decent results in my previous tests.

In [4]:
pipeline_summ = pipeline(
    "summarization",
    model="facebook/bart-large-cnn", # switch out to "t5-small" etc if you wish
    tokenizer="facebook/bart-large-cnn", # as above
    framework="pt",
)

# First of 2 summarization function
def fb_summarizer(text):
    input_text = clean_text(text)
    results = pipeline_summ(input_text)
    return results[0]["summary_text"]

# First of 2 Gradio apps that we'll put in "parallel"
summary1 = gr.Interface(
    fn=fb_summarizer,
    inputs=gr.inputs.Textbox(),
    outputs=gr.outputs.Textbox(label="Summary by FB/Bart-large"),
)


## 1.2 SUMMARIZATION VIA HUGGING FACE'S MODEL HUB

Not all transformer models finetuned for summarization are compatible with the pipeline. The relatively [newer Pegasus models by Google](https://ai.googleblog.com/2020/06/pegasus-state-of-art-model-for.html) have to be loaded up the "old-fashioned" way as below.

In [5]:
model_name = "google/pegasus-cnn_dailymail" # Pegasus has a few variations; switch out as required

# Second of 2 summarization function
def google_summarizer(text):
    input_text = clean_text(text)
    
    tokenizer_pegasus = AutoTokenizer.from_pretrained(model_name)

    model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    batch = tokenizer_pegasus.prepare_seq2seq_batch(
        input_text, truncation=True, padding="longest", return_tensors="pt"
    )
    translated = model_pegasus.generate(**batch)

    pegasus_summary = tokenizer_pegasus.batch_decode(
        translated, skip_special_tokens=True
    )

    return pegasus_summary[0]

# Second of 2 Gradio apps that we'll put in "parallel"
summary2 = gr.Interface(
    fn=google_summarizer,
    inputs=gr.inputs.Textbox(),
    outputs=gr.outputs.Textbox(label="Summary by Google/Pegasus-CNN-Dailymail"),
)


# 2 LAUNCH SUMMARIZATION MODELS IN PARALLEL

In [6]:
Parallel(
    summary1,
    summary2,
    title="Compare 2 AI Summarizers",
    inputs=gr.inputs.Textbox(lines=20, label="Paste some English text here"),
).launch()


Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7860/', None)